# Some examples

## Code

### Function concerning first examples: any decompositions as rank-one bilinear map
These function are not tested in the following of the notebook, but they are left here if anyone wants to play with them. They are not particularly well implemented but they should work.

In [1]:
"""
Translate a set of indices into an element of the vector space `W`.

EXAMPLE:
    b(0, 1, V, n, m) gives the element representing a₀b₁ in V. 
    b([1, 0], [1, 1], V, n, m) gives the element representing a₀(b₀+b₁) in V. 
"""

def _b(I, J, W, n, m):
    B = W.basis()
    if I in ZZ and J in ZZ:
        d = I + J*n
        return B[d]
    else:
        S = W()
        for i in range(n):
            a = I[i]
            if a == 0:
                continue
            for j in range(m):
                b = J[j]
                if b == 0:
                    continue
                else:
                    d = i + j*n
                    S += (a*b) * B[d]
        return S

"""
Internal function needed to compute rank-one elements.
"""
def _construct_recc(L, begin, length, F):
    if length == 0:
        L.append(begin)
    else:
        for x in F:
            construct_recc(L, begin+[x], length-1, F)

"""
Internal function needed to compute rank-one elements.
"""
def _construct(length, F):
    L = []
    for j in range(length):
        construct_recc(L, j*[F(0)]+[F(1)], length-1-j, F)
    return L

"""
Compute the rank-one elements in V.
"""
def rank_one_elements(V, n, m):
    return [_b(x, y, V, n, m) for x in _construct(n, V.base_field()) for y in construct(m, V.base_field())]

"""
Compute the rank-one symmetric elements in V.
"""
def rank_one_elements_sym(V, n):
    return [_b(x, x, V, n, n) for x in _construct(n, V.base_field())]

In [2]:
"""
Compute a basis of W containing only rank-one elements.
"""
def rank_one_basis(W, G):
    
    B = [w for w in W.basis() if w in G]
    if len(B) == W.dimension():
        return B
    else:
        for g in [x for x in G if x in W]:
            if g not in W.span(B):
                B += [g]
        return B

In [3]:
"""
The `expand_subspace` as described in Covanov paper. Compute a vector space generated by rank-one elements that 
contains the elements in `targets`.

NOTE: v0
"""
def expand_subspace(targets, n, m, W = None, k = None, upto = Infinity, G = None, G2 = None, L=[]):
    if k <= upto:
        T = targets[0].parent().span(targets)

        if W == None:
            W = T

        if G == None:
            G = rank_one_elements(W.ambient_vector_space(), n, m)
            G2 = copy(G)

        if k == None:
            k = T.dimension()

        if W.dimension() == k and W.span([x for x in G if x in W]).dimension() == k:
            L.append(W)
        else:

            for g in G2:
                if g in W:
                    continue
                G3 = copy(G2)
                G3.remove(g)
                expand_subspace(targets, n, m, W+W.span([g]), k+1, upto, G, G3, L)

"""
The `expand_subspace` as described in Covanov paper. Compute a vector space generated by rank-one elements that 
contains the elements in `targets`.

NOTE: v1
"""                
def expand_subspace1(targets, n, m, W = None, k = None, upto = Infinity, G = None, G2 = None, L=[]):
    if k <= upto:
        T = targets[0].parent().span(targets)

        if W == None:
            W = T

        if G == None:
            G = rank_one_elements(W.ambient_vector_space(), n, m)
            G2 = copy(G)

        if k == None:
            k = T.dimension()

        if W.dimension() == k and W.span([x for x in G if x in W]).dimension() == k:
            L.append(W)
        else:

            for i in range(0, len(G2)):
                g = G2[i]
                if g in W:
                    continue
                G3 = copy(G2[i+1:])
                expand_subspace1(targets, n, m, W+W.span([g]), k+1, upto, G, G3, L)
                
"""
The `expand_subspace` as described in Covanov paper. Compute a vector space generated by rank-one elements that 
contains the elements in `targets`.

NOTE: v2
"""                 
def expand_subspace2(targets, n, m, W = None, elem = None, k = None, upto = Infinity, G = None, G2 = None, L=[]):
    
    if k <= upto:
        T = targets[0].parent().span(targets)

        if W == None:
            W = T

        if G == None:
            G = rank_one_elements(W.ambient_vector_space(), n, m)
            G2 = copy(G)

        if k == None:
            k = T.dimension()

        if W.dimension() == k and W.span([x for x in G if x in W]).dimension() == k:
            L.append(W)
        elif G2 != []:
            if elem == None:
                G2 = reduction(G2, W)
            else:
                G2 = reduction(G2, W.span([elem]))
            
            for i in range(len(G2)):
                g = G2[i]
                G3 = copy(G2[i+1:])
                expand_subspace2(targets, n, m, W+W.span([g]), g, k+1, upto, G, G3, L)

"""
Compute formulas to decompose the elements in `T` in rank-one elements. 
"""                  
def formulas(T, W, G):
    B = rank_one_basis(W, G)
    WW = W.span_of_basis(B)
    return B, [(t, WW.coordinates(t)) for t in T]

In [4]:
"""
Reduction of the elements in `G` modulo `W`.
""" 
def reduction(G, W):
    H = []
    B = W.echelonized_basis()
    M = W.echelonized_basis_matrix()
    P = M.pivots()
    for g in G:
        w = g.list_from_positions(P)
        h = g
        for j in range(len(w)):
            h -= w[j] * B[j]
        H.append(h)
    
    H.sort()
    I = [H[0]]
    for j in range(1, len(H)):
        if H[j-1] == H[j]:
            continue
        else:
            I.append(H[j])
    if I[0] == 0:
        I.pop(0)
    return I

"""
Reduction of the elements in `G` modulo `W`.
""" 
def reduction2(G, W):
    l = len(G)
    delete = []
    for i in range(l):
        for j in range(i+1, l):
            if (G[i]-G[j]) in W:
                delete.append(j)
    
    G2 = copy([G[i] for i in range(l) if i not in delete])
    return G2

### Function concerning tri-symmetric decomposition
First are the function related to the "vector space strategy".

In [5]:
"""
Compute a list `L` of lists of up to `upto` elements in `B` whose associated trace
bilinear form span a vector space containing `t`.
""" 
# À changer !!! On peut sûrement faire moins de calculs
def find_decomposition(t, B, upto, d, L, k = 0, G = [], W = None):
    if k <= upto:
        if W == None:
            W = t.parent().span([])
        if t in W:
            L.append(G)
        elif k < upto:
            for j in range(len(B)):
                B2 = B[j+1:]
                G2 = copy(G)+[B[j]]
                find_decomposition(t, B2, upto, d, L, k+1, G2, W+t.parent().span([d[B[j]]]))

"""
Compute the bilinear map associated to `x`, represented by an element in `V`.
""" 
def trace_to_bil(x, V):
    n = sqrt(V.dimension())
    g = x.parent().gen()
    L = [(x * g^j).trace() for j in range(n)]
    return _b(L, L, V, n, n)

"""
Compute a dictionnary associating elements of `k` with their bilinear form in `V`.
""" 
def make_dict_global(k, V):
    d = dict()
    for x in k:
        d[x] = trace_to_bil(x, V)
    return d

"""
Compute the elements in `k` whose `j`-th coordinate in the canonical basis is 1, and the i-th coordinates
for i<j are zero.
""" 
def fields_elements(k, j):
    L = []
    for x in k:
        y = x.polynomial().coefficients(sparse=False)
        if (len(y) >= j+1) and (y[j] == 1):
            boo = True
            for l in range(j):
                if y[l] != 0:
                    boo = False
                    break
            if boo:
                L.append(x)
    return L

In [6]:
"""
Returns True if `T` is a list of zero elements. False otherwise.
""" 
def finished(T):
    for i in range(len(T)):
        if T[i] != 0:
            return False
    return True

"""
Compute the tri-symmetric decompositions of `T` 

NOTE:
    Apply the `find_decomposition` strategy in each coordinate of `T` searching for solutions with only 
    `upto_each` rank-one elements, while `upto` rank-one elements are allowed in the whole process.
""" 
def tri_symmetric_search(T, j, k, V, d, upto, upto_each, Lglob, Lloc = []):
    if finished(T):
        Lloc.sort()
        Lglob.append(Lloc)
    else:
        u = min(upto_each, upto) # how many products we can still use
        M = []
        find_decomposition(T[j], fields_elements(k, j), u, d, M)
        if len(M) > 0:
            for m in M:
                B = [d[s] for s in m]
                W = V.subspace_with_basis(B)
                coord = W.coordinates(T[j])
                S = copy(T)
                Lloc2 = copy(Lloc)
                for i in range(len(coord)):
                    Lloc2.append((m[i], coord[i]))
                    P = m[i].polynomial()
                    for l in range(j, P.degree()+1):
                        S[l] = S[l] - P[l]*coord[i]*d[m[i]]
                tri_symmetric_search(S, j+1, k, V, d, upto-len(m), upto_each, Lglob, Lloc2)        

In [7]:
"""
Compute the list of vector of `V` associated with the multiplication bilinear form in `k`.
""" 
def multiplication_formula(k, V):
    P = k.modulus()
    n = k.degree()
    T = (2*n-1)*[V()]
    for j in range(n):
        for i in range(n):
            T[i+j] += _b(j, i, V, n, n)
    
    l = len(T)
    
    while l > n:
        for j in range(P.degree()+1):
            T[l-1-n+j] -= P[j]*T[l-1]
        l -= 1
        T.pop(-1)
    
    return T

### Functions concerning Frobenius-fixed solutions

In [8]:
def normalized_frobenius(x):
    f = x.parent().frobenius_endomorphism()
    y = f(x)
    P = y.polynomial()
    j = 0

    while P[j] == 0:
        j += 1

    e = P[j]
    return y/e

def normalized_frobenius_orbit(x):
    orbit = [x]
    conj = normalized_frobenius(x)
    while not conj in orbit:
        orbit.append(conj)
        conj = normalized_frobenius(conj)
    return orbit

def normalized_frobenius_tuple(x):
    a, c = x
    f = a.parent().frobenius_endomorphism()
    b = f(a)
    P = b.polynomial()
    j = 0

    while P[j] == 0:
        j += 1

    e = P[j]
    return (b/e, c*e)

def normalized_frobenius_tuple_orbit(x):
    orbit = [x]
    conj = normalized_frobenius_tuple(x)
    while not conj in orbit:
        orbit.append(conj)
        conj = normalized_frobenius(conj)
    return orbit

def list_frobenius(L):
    M = [normalized_frobenius_tuple(x) for x in L]
    M.sort()
    return M

def list_frobenius_orbit(L):
    orbit = [L]
    conj = list_frobenius(L)
    while not conj in orbit:
        orbit.append(conj)
        conj = list_frobenius(conj)
    return orbit

In [9]:
def first_nonzero_coord(x):
    y = x.polynomial()
    j = 0
    while y[j] == 0:
        j += 1
    return j

def find_decomposition_frob(t, elems_global, index, upto, d, L, k = 0, G = [], W = None):
    if k <= upto:
        if W == None:
            W = t.parent().span([])
        if t in W:
            L.append(G)
        else:
            j = 0
            while B != []:
                b = B[0]
                orbit = normalized_frobenius((b, b.parent().one()))
                elems_in_orbit = [a for a,c in orbit]
                for elem in elems_in_orbit:
                    elems_global[firs_nonzero_coord(elem)].append(elem)
                
            
            
            1
            for j in range(len(B)):
                B2 = B[j+1:]
                G2 = copy(G)+[B[j]]
                find_decomposition(t, elems_global, index, upto, d, L, k+1, G2, W+t.parent().span([d[B[j]]]))

def finished_exhaustive_search(T, L, M, V, d):
    n = len(T)
    S = copy(T)
    elems = []
    for j in range(n):
        elems += [[]]
    for a in L:
        index = first_nonzero_coord(a)
        elems[index].append(a)
    for j in range(n):
        m = elems[j]
        try:
            W = V.subspace_with_basis([d[x] for x in m])
        except ValueError:
            return False
            
        if S[j] in W:
            coord = W.coordinates(S[j])
            for i in range(len(coord)):
                M.append((m[i], coord[i]))
                P = m[i].polynomial()
                for l in range(j, P.degree()+1):
                    S[l] = S[l] - P[l]*coord[i]*d[m[i]]
        else:
            return False

        
    return True
    
def tri_symmetric_frob_exhaustive_search(T, k, V, d, upto, Lglob, count = 0, orbits = None, chosen_orbits = []):
    
    if count <= upto:
        M = []
        
        if finished_exhaustive_search(T, chosen_orbits, M, V, d):

            M.sort()
            Lglob.append(M)

        elif count < upto:        
    
            if orbits == None:
                orbits = normalized_orbits(k)

            for j in range(len(orbits)):
                new_chosen_orbits = chosen_orbits+orbits[j]
                remaining_orbits = copy(orbits[j+1:])
                tri_symmetric_frob_exhaustive_search(T, k, V, d, upto, Lglob, count+len(orbits[j]), remaining_orbits, new_chosen_orbits)      

In [10]:
def normalized_orbits(k):
    n = k.degree()
    elems = [fields_elements(k, j) for j in range(n)]
    orbits = []
    j = 0
    while j < n:
        while elems[j] != []:
            x = elems[j][0]
            orb = normalized_frobenius_orbit(x)
            orbits.append(orb)
            for a in orb:
                index = first_nonzero_coord(a)
                elems[index].remove(a)
        j += 1
    return orbits

### Functions using matrix representation
There are different versions of the matrix strategy.
* the first one, without label, implements the "naive" way
* the ones labeled `3` where a first prototype of a non-naive version
* the ones labeled `4` are the non-naive version with "margins"
* the ones labeled `4bis` are a slightly different version of `4` that were used to test how Python works

In [11]:
def matrix_translation(I, J, S):
    n, m = S.dims()
    M = S()
    if I in ZZ and J in ZZ:
        M[I, J] = 1
        return M
    else:
        M = S()
        for i in range(n):
            a = I[i]
            if a == 0:
                continue
            for j in range(m):
                b = J[j]
                if b == 0:
                    continue
                else:
                    M[i, j] = a*b
        return M
    
"""
Compute the bilinear map associated to `x`, represented by an element in `S`.
""" 
def trace_to_bil_mat(x, S):
    n = S.ncols()
    g = x.parent().gen()
    L = [(x * g^j).trace() for j in range(n)]
    return matrix_translation(L, L, S)

"""
Compute a dictionnary associating elements of `k` with their bilinear form in `S`.
""" 
def make_dict_global_mat(k, S):
    d = dict()
    for x in k:
        d[x] = trace_to_bil_mat(x, S)
    return d

In [12]:
"""
Compute the list of vector of `V` associated with the multiplication bilinear form in `k`.
""" 
def multiplication_formula_mat(k, S):
    P = k.modulus()
    n = k.degree()
    T = (2*n-1)*[S()]
    for j in range(n):
        for i in range(n):
            T[i+j] += matrix_translation(j, i, S)
    
    l = len(T)
    
    while l > n:
        for j in range(P.degree()+1):
            T[l-1-n+j] -= P[j]*T[l-1]
        l -= 1
        T.pop(-1)
    
    return T

In [13]:
"""
Compute a list `L` of lists of elements that form a minimal decomposition 
of the matrix `t`.
"""
def find_decomposition_mat(t, B, d, L, G = []):
    if t.rank() == 0:
        L.append(G)
    else:
        r = t.rank()
        Field = t.base_ring()
        #nonZeroElems = [x for x in Field if x != 0]
        nonZeroElems = list(Field)[1:]
        elems = []
        for b in B:
            for x in nonZeroElems:
                if (t - x*d[b]).rank() < r:
                    elems.append((b, x))
                    break
        for j in range(len(elems)):
            y = elems[j]
            B2 = [elems[i][0] for i in range(j+1, len(elems))]
            find_decomposition_mat(t-y[1]*d[y[0]], B2, d, L, copy(G+[y]))

"""
Compute a list `Lglob` of tri-symmetric decompositions, where for each coordinate
the search is minimal.  
"""
def tri_symmetric_search_mat(T, k, d, Lglob, bound = Infinity, j = 0, Lloc = []):
    
    if finished(T):
        Lloc.sort()
        Lglob.append(Lloc)
    elif T[j].rank() <= bound:
        M = []
        find_decomposition_mat(T[j], fields_elements(k, j), d, M)

        for m in M:
            T2 = copy(T)
            T2[j] = 0
            Lloc2 = Lloc+m
            for x in m:
                P = x[0].polynomial()
                for l in range(j+1, P.degree()+1):
                    T2[l] = T2[l] - P[l]*x[1]*d[x[0]]
            
            tri_symmetric_search_mat(T2, k, d, Lglob, bound-len(m), j+1, Lloc2)

## Some other tests
Where we perform less rank tests by previously limitating the set of candidates. Unfortunately the time used to eliminate the candidates turns out to be non negligible and this strategy is not efficient.

In [14]:
"""
Return a random rank one element in `matrix_space`.
"""
def random_rank_one_element(matrix_space):
    dimension = matrix_space.ncols()
    base_ring = matrix_space.base_ring()
    vector_space = MatrixSpace(k, dimension, 1)
    y = vector_space.random_element()
    while y.is_zero():
        y = vector_space.random_element()
        
    return matrix_space(y*y.transpose())

"""
Compute a one-line matrix correspondind to the trace form of `field`
over its base field.
"""
def trace_form(field):
    base_field = field.base_ring()
    deg = field.degree()
    matrixSpace = MatrixSpace(base_field, 1, deg)
    gen = field.gen()
    trace = matrixSpace([(gen^j).trace() for j in range(deg)])
    return trace

"""
Compute a the multiplication-by-`element` matrix.
"""
def multiplication_matrix(element):
    field = element.parent()
    return element.polynomial()(companion_matrix(field.modulus()))

"""
Compute a the vector X such that X × transpose(X) is the matrix corresponding
to the map z |-> trace(`element` × z).
"""
def corresponding_vector(element):
    field = element.parent()
    vector_space = (field.base_ring())^field.degree()
    vector = vector_space((trace_form(field)*multiplication_matrix(element)).list())
    return vector

"""
Compute a dictionnary of elements in `field` and their corresponding vectors.
"""
def make_vectors(field):
    vectors = dict()
    for a in field:
        vectors[a] = corresponding_vector(a)
    return vectors

"""
Compute a list `L` of lists of elements that form a minimal decomposition 
of the matrix `t`.

# Notes:
    Needs the dictionnary `vectors`.
"""
def find_decomposition_mat2(t, B, d, vectors, L, G = []):
    global CPT, CPT2
    if t.rank() == 0:
        L.append(G)
    else:
        r = t.rank()
        Field = t.base_ring()
        nonZeroElems = [x for x in Field if x != 0]
        elems = []
        Img = t.image()
        for b in B:
            if vectors[b] in Img:
                CPT2 += 1
                for x in nonZeroElems:
                    if (t - x*d[b]).rank() < r:
                        CPT += 1
                        elems.append((b, x))
                        break
        for j in range(len(elems)):
            y = elems[j]
            B2 = [elems[i][0] for i in range(j+1, len(elems))]
            find_decomposition_mat2(t-y[1]*d[y[0]], B2, d, vectors, L, copy(G+[y]))
            
"""
Compute a list `Lglob` of tri-symmetric decompositions, where for each coordinate
the search is minimal.

# Notes:
    Needs the dictionnary `vectors`.  
"""
def tri_symmetric_search_mat2(T, k, d, vectors, Lglob, bound = Infinity, j = 0, Lloc = []):
    if finished(T):
        Lloc.sort()
        Lglob.append(Lloc)
    elif T[j].rank() <= bound:
        M = []
        find_decomposition_mat2(T[j], fields_elements(k, j), d, vectors, M)

        for m in M:
            T2 = copy(T)
            T2[j] = 0
            Lloc2 = Lloc+m
            for x in m:
                P = x[0].polynomial()
                for l in range(j+1, P.degree()+1):
                    T2[l] = T2[l] - P[l]*x[1]*d[x[0]]
            
            tri_symmetric_search_mat2(T2, k, d, vectors, Lglob, bound-len(m), j+1, Lloc2)

In [15]:
def compute_search_set(target, fields_elems, count, dictionnary, global_set, local_set = [], index = 0):
    if count == 0:
        global_set.append((local_set, index, target))
    else:
        Field = target.base_ring()
        nonZeroElems = list(Field)[1:]
        for j in range(index, len(fields_elems)):
            elem = fields_elems[j]
            for x in nonZeroElems:
                new_target = target-x*dictionnary[elem]
                compute_search_set(new_target, fields_elems, count-1, dictionnary, global_set, local_set+[(elem, x)], j+1)

def find_decomposition_mat3(t, B, d, L, begin = [], G = []):
    if t.rank() == 0:
        L.append(begin+G)
    else:
        r = t.rank()
        Field = t.base_ring()
        nonZeroElems = list(Field)[1:]
        elems = []
        for b in B:
            for x in nonZeroElems:
                if (t - x*d[b]).rank() < r:
                    elems.append((b, x))
                    break
        for j in range(len(elems)):
            y = elems[j]
            B2 = [elems[i][0] for i in range(j+1, len(elems))]
            find_decomposition_mat3(t-y[1]*d[y[0]], B2, d, L, begin, copy(G+[y]))

def tri_symmetric_search_mat3(T, k, d, Lglob, counts = None, bound = Infinity, j = 0, Lloc = []):
    if counts == None:
        counts = len(T)*[0]
    
    if finished(T):
        Lloc.sort()
        Lglob.append(Lloc)
    elif T[j].rank() <= bound:
        
        M = []
        search_set = []
        fields_elems = fields_elements(k, j)
        count = counts[j]
        compute_search_set(T[j], fields_elems, count, d, search_set)

        for t in search_set:
            find_decomposition_mat3(t[2], fields_elems[t[1]:], d, M, t[0])
        
        for m in M:
            T2 = copy(T)
            T2[j] = 0
            Lloc2 = Lloc+m
            for x in m:
                P = x[0].polynomial()
                for l in range(j+1, P.degree()+1):
                    T2[l] = T2[l] - P[l]*x[1]*d[x[0]]
            
            tri_symmetric_search_mat3(T2, k, d, Lglob, counts, bound-len(m), j+1, Lloc2)

In [16]:
def find_decomposition_mat4(t, B, d, L, count, bound, G = []):
    r = t.rank()
    if r <= bound:
        if r == 0:
            L.append(G)
        elif count == 0:
            Field = t.base_ring()
            #nonZeroElems = [x for x in Field if x != 0]
            nonZeroElems = list(Field)[1:]
            elems = []
            for b in B:
                for x in nonZeroElems:
                    if (t - x*d[b]).rank() < r:
                        elems.append((b, x))
                        break
            for j in range(len(elems)):
                y = elems[j]
                B2 = [elems[i][0] for i in range(j+1, len(elems))]
                find_decomposition_mat4(t-y[1]*d[y[0]], B2, d, L, count, bound-1, copy(G+[y]))
        else:
            Field = t.base_ring()
            #nonZeroElems = [x for x in Field if x != 0]
            nonZeroElems = list(Field)[1:]
            for j in range(len(B)):
                b = B[j]
                B2 = B[j+1:]
                for x in nonZeroElems:
                    t2 = t - x*d[b]
                    r2 = t2.rank()
                    if r2 < r:                    
                        find_decomposition_mat4(t2, B2, d, L, count, bound-1, copy(G+[(b, x)]))
                    elif r2 == r: #else what happens if we allow the rank to rise? ie r2 > r?
                        find_decomposition_mat4(t2, B2, d, L, count-1, bound-1, copy(G+[(b, x)]))

"""
Compute a list `Lglob` of tri-symmetric decompositions, where for each coordinate
the search is minimal.  
"""
def tri_symmetric_search_mat4(T, k, d, Lglob, counts = None, bound = Infinity, j = 0, Lloc = []):

    if counts == None:
        counts = len(T)*[0]
        
    if finished(T):
        Lloc.sort()
        Lglob.append(Lloc)
    elif T[j].rank() <= bound:
        M = []
        count = counts[j] # min(count[j], bound) smth like that?
        find_decomposition_mat4(T[j], fields_elements(k, j), d, M, count, bound)

        for m in M:
            T2 = copy(T)
            T2[j] = 0
            Lloc2 = Lloc+m
            for x in m:
                P = x[0].polynomial()
                for l in range(j+1, P.degree()+1):
                    T2[l] = T2[l] - P[l]*x[1]*d[x[0]]
            
            tri_symmetric_search_mat4(T2, k, d, Lglob, counts, bound-len(m), j+1, Lloc2)

In [17]:
def find_decomposition_mat4bis(t, B, d, L, count, bound, G = [], index = 0):
    r = t.rank()
    if r <= bound:
        if r == 0:
            L.append(G)
        elif count == 0:
            Field = t.base_ring()
            #nonZeroElems = [x for x in Field if x != 0]
            nonZeroElems = list(Field)[1:]
            elems = []
            for j in range(index, len(B)):
                b = B[j]
                for x in nonZeroElems:
                    if (t - x*d[b]).rank() < r:
                        elems.append((b, x))
                        break
            for j in range(len(elems)):
                y = elems[j]
                B2 = [elems[i][0] for i in range(j+1, len(elems))]
                find_decomposition_mat4bis(t-y[1]*d[y[0]], B2, d, L, count, bound-1, copy(G+[y]))
        else:
            Field = t.base_ring()
            #nonZeroElems = [x for x in Field if x != 0]
            nonZeroElems = list(Field)[1:]
            for j in range(index, len(B)):
                b = B[j]
                for x in nonZeroElems:
                    t2 = t - x*d[b]
                    r2 = t2.rank()
                    if r2 < r:                    
                        find_decomposition_mat4bis(t2, B, d, L, count, bound-1, copy(G+[(b, x)]), j+1)
                    elif r2 == r: #else what happens if we allow the rank to rise? ie r2 > r?
                        find_decomposition_mat4bis(t2, B, d, L, count-1, bound-1, copy(G+[(b, x)]), j+1)

"""
Compute a list `Lglob` of tri-symmetric decompositions, where for each coordinate
the search is minimal.  
"""
def tri_symmetric_search_mat4bis(T, k, d, Lglob, counts = None, bound = Infinity, j = 0, Lloc = []):
    if counts == None:
        counts = len(T)*[0]
        
    if finished(T):
        Lloc.sort()
        Lglob.append(Lloc)
    elif T[j].rank() <= bound:
        M = []
        count = counts[j] # min(count[j], bound) smth like that?
        find_decomposition_mat4bis(T[j], fields_elements(k, j), d, M, count, bound)

        for m in M:
            T2 = copy(T)
            T2[j] = 0
            Lloc2 = Lloc+m
            for x in m:
                P = x[0].polynomial()
                for l in range(j+1, P.degree()+1):
                    T2[l] = T2[l] - P[l]*x[1]*d[x[0]]
            
            tri_symmetric_search_mat4bis(T2, k, d, Lglob, counts, bound-len(m), j+1, Lloc2)
            

In [18]:
def test_algs(T, k, d, counts = None, bound = Infinity):
    
    print "*** Algo 1 ***"
    L = []
    %time tri_symmetric_search_mat(T, k, d, L, bound)
    
    print "*** Algo 3 ***"
    L3 = []
    %time tri_symmetric_search_mat3(T, k, d, L3, counts, bound)
    
    print "*** Algo 4 ***"
    L4 = []
    %time tri_symmetric_search_mat4(T, k, d, L4, counts, bound)
    
    print "*** Algo 4bis ***"
    L4bis = []
    %time tri_symmetric_search_mat4bis(T, k, d, L4bis, counts, bound)
    
    return L, L3, L4, L4bis

# Tri-symmetric study of $\mathbb F_9/\mathbb F_3$

In [58]:
k = GF(9)
n, m = 2, 2
V = VectorSpace(GF(3), n*m)
d = make_dict_global(k, V)
T = multiplication_formula(k, V)

In [59]:
L = []
tri_symmetric_search(T, 0, k, V, d, 3, 3, L)
L

[[(1, 2), (z2, 2), (2*z2 + 1, 2)]]

# Tri-symmetric study of $\mathbb F_{27}/\mathbb F_3$

In [17]:
n, m = 3, 3
k = GF(27)
V = VectorSpace(GF(3), n*m)
d = make_dict_global(k, V)
T = multiplication_formula(k, V)
k.modulus()

x^3 + 2*x + 1

In [18]:
M = []
%time tri_symmetric_search(T, 0, k, V, d, 6, 3, M)
M

CPU times: user 440 ms, sys: 3.99 ms, total: 444 ms
Wall time: 442 ms


[[(z3, 1),
  (z3 + 1, 1),
  (2*z3 + 1, 2),
  (z3^2 + z3, 2),
  (2*z3^2 + 1, 1),
  (2*z3^2 + z3, 1)],
 [(z3^2, 1),
  (z3^2 + 1, 1),
  (z3^2 + z3, 2),
  (z3^2 + 2*z3 + 1, 2),
  (2*z3^2 + z3, 2),
  (2*z3^2 + z3 + 1, 1)]]

# Tri-symmetric study of $\mathbb F_{3^4}/\mathbb F_3$

In [21]:
k = GF(3^4)
n, m = 4, 4
V = VectorSpace(GF(3), n*m)
d = make_dict_global(k, V)
T = multiplication_formula(k, V)

In [221]:
L = []
%time tri_symmetric_search(T, 0, k, V, d, 9, 4, L)
len(L)

CPU times: user 2min 42s, sys: 1.15 s, total: 2min 43s
Wall time: 2min 44s


5

In [222]:
L[0]

[(z4, 2),
 (z4^2, 2),
 (z4^3, 2),
 (z4^3 + 1, 2),
 (z4^3 + z4, 2),
 (z4^3 + z4^2, 1),
 (z4^3 + 2*z4^2 + 1, 2),
 (2*z4^3 + z4^2 + z4 + 1, 1),
 (2*z4^3 + 2*z4^2 + 2*z4 + 1, 2)]

# Tri-symmetric study of $\mathbb F_{3^5}/\mathbb F_3$

In [9]:
n = 5
k = GF(3^n)
x = gen(k)
V = VectorSpace(GF(3), n^2)
d = make_dict_global(k, V)
T = multiplication_formula(k, V)

In [3]:
L = []
#%time tri_symmetric_search(T, 0, k, V, d, 11, 5, L)
#L
# OUT OF TIME!

# Playing with automorphisms

# $\mathbb F_{3^3}/\mathbb{F}_3$

In [22]:
k = GF(27)
n, m = 3, 3
V = VectorSpace(GF(3), n*m)
d = make_dict_global(k, V)
T = multiplication_formula(k, V)

In [23]:
L = []
%time tri_symmetric_search(T, 0, k, V, d, 6, 6, L)
L

CPU times: user 1.63 s, sys: 24.8 ms, total: 1.65 s
Wall time: 1.64 s


[[(z3^2 + z3, 1),
  (z3^2 + z3 + 1, 2),
  (z3^2 + 2*z3 + 1, 1),
  (2*z3^2 + 1, 1),
  (2*z3^2 + z3 + 1, 2),
  (2*z3^2 + 2*z3 + 1, 1)],
 [(z3^2, 2),
  (z3^2 + 1, 2),
  (z3^2 + z3 + 1, 1),
  (2*z3^2 + 1, 2),
  (2*z3^2 + z3, 1),
  (2*z3^2 + 2*z3 + 1, 2)],
 [(z3, 1),
  (z3 + 1, 1),
  (2*z3 + 1, 2),
  (z3^2 + z3, 2),
  (2*z3^2 + 1, 1),
  (2*z3^2 + z3, 1)],
 [(z3^2, 1),
  (z3^2 + 1, 1),
  (z3^2 + z3, 2),
  (z3^2 + 2*z3 + 1, 2),
  (2*z3^2 + z3, 2),
  (2*z3^2 + z3 + 1, 1)]]

In [24]:
M =[]
%time tri_symmetric_frob_exhaustive_search(T, k, V, d, 6, M)
M

CPU times: user 142 ms, sys: 3.63 ms, total: 146 ms
Wall time: 150 ms


[[(z3, 1),
  (z3 + 1, 1),
  (2*z3 + 1, 2),
  (z3^2 + z3, 2),
  (2*z3^2 + 1, 1),
  (2*z3^2 + z3, 1)]]

# $\mathbb F_{3^4}$

In [25]:
p, n = 3, 4
k = GF(p^n)
V = VectorSpace(GF(p), n^2)
d = make_dict_global(k, V)
T = multiplication_formula(k, V)

In [27]:
L = []
%time tri_symmetric_search(T, 0, k, V, d, 9, 4, L)
len(L)

CPU times: user 2min 54s, sys: 1.28 s, total: 2min 55s
Wall time: 2min 55s


In [26]:
M =[]
%time tri_symmetric_frob_exhaustive_search(T, k, V, d, 9, M)
M

CPU times: user 871 ms, sys: 12.6 ms, total: 883 ms
Wall time: 865 ms


[[(1, 2),
  (z4, 2),
  (z4^2 + z4 + 1, 1),
  (2*z4^2 + z4, 2),
  (z4^3, 2),
  (z4^3 + 2*z4^2 + 1, 2),
  (z4^3 + 2*z4^2 + 2*z4 + 1, 2),
  (2*z4^3 + z4, 2),
  (2*z4^3 + 2*z4^2 + z4, 1)]]

In [28]:
for l in L:
    print len(list_frobenius_orbit(l)) == 1

False
False
False
False
True


In [30]:
L[-1] == M[0]

True

# $\mathbb F_{3^5}$

In [27]:
p, n = 3, 5
k = GF(p^n)
V = VectorSpace(GF(p), n^2)
d = make_dict_global(k, V)
T = multiplication_formula(k, V)

In [28]:
M =[]
%time tri_symmetric_frob_exhaustive_search(T, k, V, d, 11, M)
M

CPU times: user 1.58 s, sys: 24.7 ms, total: 1.6 s
Wall time: 1.58 s


[[(1, 1),
  (2*z5 + 1, 2),
  (2*z5^2 + 2*z5 + 1, 1),
  (z5^3 + 2*z5 + 1, 1),
  (2*z5^3 + 1, 2),
  (2*z5^3 + 2*z5^2 + z5 + 1, 1),
  (2*z5^4 + z5 + 1, 1),
  (2*z5^4 + 2*z5^2 + 1, 1),
  (2*z5^4 + 2*z5^3 + z5, 1),
  (2*z5^4 + 2*z5^3 + z5^2 + 1, 2),
  (2*z5^4 + 2*z5^3 + z5^2 + z5 + 1, 1)]]

# $\mathbb F_{3^6}$

In [42]:
p, n = 3, 6
k = GF(p^n)
V = VectorSpace(GF(p), n^2)
d = make_dict_global(k, V)
T = multiplication_formula(k, V)

In [40]:
M =[]
%time tri_symmetric_frob_exhaustive_search(T, k, V, d, 15, M)
M

CPU times: user 2min 14s, sys: 472 ms, total: 2min 14s
Wall time: 2min 14s


[[(2*z6^4 + 1, 1),
  (2*z6^4 + 2*z6^2 + z6 + 1, 1),
  (2*z6^4 + z6^3 + 1, 2),
  (2*z6^4 + z6^3 + z6 + 1, 1),
  (z6^5 + z6^3 + 1, 2),
  (z6^5 + z6^3 + 2*z6^2 + 2*z6 + 1, 1),
  (z6^5 + 2*z6^3 + 1, 1),
  (z6^5 + z6^4 + z6^3 + 2*z6 + 1, 1),
  (z6^5 + z6^4 + 2*z6^3 + z6, 1),
  (z6^5 + z6^4 + 2*z6^3 + 2*z6^2 + 2*z6 + 1, 2),
  (z6^5 + 2*z6^4 + z6^3 + 2*z6^2 + 2*z6 + 1, 1),
  (z6^5 + 2*z6^4 + 2*z6^3 + 1, 1),
  (z6^5 + 2*z6^4 + 2*z6^3 + z6 + 1, 2),
  (2*z6^5 + z6^2 + 2*z6 + 1, 1),
  (2*z6^5 + z6^4 + z6^3 + 2*z6^2 + 2*z6 + 1, 2)],
 [(2*z6^4 + z6^2 + z6 + 1, 2),
  (z6^5 + z6 + 1, 1),
  (z6^5 + z6^3 + z6^2 + z6 + 1, 2),
  (z6^5 + z6^4, 2),
  (z6^5 + z6^4 + 1, 2),
  (z6^5 + 2*z6^4 + z6, 2),
  (2*z6^5 + z6^3 + z6 + 1, 2),
  (2*z6^5 + z6^3 + 2*z6 + 1, 2),
  (2*z6^5 + z6^3 + 2*z6^2 + 1, 1),
  (2*z6^5 + z6^3 + 2*z6^2 + z6, 1),
  (2*z6^5 + 2*z6^4 + 1, 1),
  (2*z6^5 + 2*z6^4 + z6^2 + z6, 1),
  (2*z6^5 + 2*z6^4 + z6^3 + z6^2, 1),
  (2*z6^5 + 2*z6^4 + 2*z6^3 + z6^2 + z6, 1),
  (2*z6^5 + 2*z6^4 + 2*z6^3 + 2

In [41]:
len(M[0]), len(M[1])

(15, 15)

# $\mathbb F_{3^7}$

In [15]:
p, n = 3, 7
k = GF(p^n)
V = VectorSpace(GF(p), n^2)
d = make_dict_global(k, V)
T = multiplication_formula(k, V)

In [16]:
M =[]
%time tri_symmetric_frob_exhaustive_search(T, k, V, d, 20, M)
M

CPU times: user 1min 23s, sys: 199 ms, total: 1min 23s
Wall time: 1min 23s


[]

# $\mathbb F_{3^8}$

In [20]:
p, n = 3, 8
k = GF(p^n)
V = VectorSpace(GF(p), n^2)
d = make_dict_global(k, V)
T = multiplication_formula(k, V)

In [23]:
M = []
%time tri_symmetric_frob_exhaustive_search(T, k, V, d, 15, M) # Long enough...
M

CPU times: user 7min 42s, sys: 1.12 s, total: 7min 43s
Wall time: 7min 43s


[]

# Things with matrices

In [69]:
p, n = 3, 3
k = GF(p^n)
S = MatrixSpace(GF(p), n, n)
d = make_dict_global_mat(k, S)
vectors = make_vectors(k)
T = multiplication_formula_mat(k, S)

In [72]:
L, L3, L4, L4bis = test_algs(T, k, d, counts = [2, 1, 0], bound = 6)

*** Algo 1 ***
CPU times: user 33.7 ms, sys: 68 µs, total: 33.8 ms
Wall time: 45.6 ms
*** Algo 3 ***
CPU times: user 155 ms, sys: 88 µs, total: 155 ms
Wall time: 162 ms
*** Algo 4 ***
CPU times: user 39.4 ms, sys: 3.95 ms, total: 43.4 ms
Wall time: 41.1 ms
*** Algo 4bis ***
CPU times: user 34.5 ms, sys: 0 ns, total: 34.5 ms
Wall time: 33.9 ms


In [73]:
L4

[[(z3^2, 2),
  (z3^2 + 1, 2),
  (z3^2 + z3 + 1, 1),
  (2*z3^2 + 1, 2),
  (2*z3^2 + z3, 1),
  (2*z3^2 + 2*z3 + 1, 2)],
 [(z3^2 + z3, 1),
  (z3^2 + z3 + 1, 2),
  (z3^2 + 2*z3 + 1, 1),
  (2*z3^2 + 1, 1),
  (2*z3^2 + z3 + 1, 2),
  (2*z3^2 + 2*z3 + 1, 1)],
 [(z3, 1),
  (z3 + 1, 1),
  (2*z3 + 1, 2),
  (z3^2 + z3, 2),
  (2*z3^2 + 1, 1),
  (2*z3^2 + z3, 1)],
 [(z3^2, 1),
  (z3^2 + 1, 1),
  (z3^2 + z3, 2),
  (z3^2 + 2*z3 + 1, 2),
  (2*z3^2 + z3, 2),
  (2*z3^2 + z3 + 1, 1)]]

In [39]:
len(L), len(L3), len(L4), L4 == L4bis

(1, 3, 4, True)

### Remark about this computation

We find only **one** solution, while the other method finds **two** solutions if you allow to search for solutions with 3 elements in each coordinate. If finds **four** solutions if you allow any type of solutions.

What is happening here is that one of the solution requires 3 terms in order to nullify a rank 2 matrix, and this is not seen by that method.

### Matrices with $\mathbb F_{3^4}$

In [25]:
p, n = 3, 4
k = GF(p^n)
S = MatrixSpace(GF(p), n, n)
d = make_dict_global_mat(k, S)
T = multiplication_formula_mat(k, S)

In [26]:
L1, L3, L4, L4bis = test_algs(T, k, d, counts = [1, 1, 0, 0], bound = 9)

*** Algo 1 ***
CPU times: user 268 ms, sys: 20.1 ms, total: 288 ms
Wall time: 262 ms
*** Algo 3 ***
CPU times: user 422 ms, sys: 32.3 ms, total: 454 ms
Wall time: 415 ms
*** Algo 4 ***
CPU times: user 525 ms, sys: 40.3 ms, total: 565 ms
Wall time: 516 ms
*** Algo 4bis ***
CPU times: user 525 ms, sys: 40.9 ms, total: 566 ms
Wall time: 517 ms


In [27]:
len(L1), len(L3), len(L4), L4 == L4bis

(2, 8, 10, True)

In [44]:
L1

[[(z4, 2),
  (z4^2, 2),
  (z4^3, 2),
  (z4^3 + 1, 2),
  (z4^3 + z4, 2),
  (z4^3 + z4^2, 1),
  (z4^3 + 2*z4^2 + 1, 2),
  (2*z4^3 + z4^2 + z4 + 1, 1),
  (2*z4^3 + 2*z4^2 + 2*z4 + 1, 2)],
 [(z4, 2),
  (z4^2, 2),
  (z4^3, 2),
  (z4^3 + z4^2 + z4 + 1, 2),
  (z4^3 + z4^2 + 2*z4 + 1, 2),
  (z4^3 + 2*z4^2 + z4, 1),
  (2*z4^3 + z4 + 1, 2),
  (2*z4^3 + 2*z4^2 + 1, 1),
  (2*z4^3 + 2*z4^2 + z4, 1)]]

### Matrices with $\mathbb F_{3^5}$

In [19]:
p, n = 3, 5
k = GF(p^n)
S = MatrixSpace(GF(p), n, n)
d = make_dict_global_mat(k, S)
vectors = make_vectors(k)
T = multiplication_formula_mat(k, S)

In [20]:
L1, L3, L4, L4bis = test_algs(T, k, d, counts = [0, 1, 1, 0, 0], bound = 11)

*** Algo 1 ***
CPU times: user 1.78 s, sys: 102 ms, total: 1.89 s
Wall time: 1.77 s
*** Algo 3 ***
CPU times: user 11.3 s, sys: 245 ms, total: 11.6 s
Wall time: 11.2 s
*** Algo 4 ***
CPU times: user 11.6 s, sys: 180 ms, total: 11.8 s
Wall time: 11.5 s
*** Algo 4bis ***
CPU times: user 11.7 s, sys: 161 ms, total: 11.9 s
Wall time: 11.6 s


In [19]:
len(L1), len(L3), len(L4), len(L4bis)

(0, 3, 0, 0)

In [24]:
len(L3[2])

13

In [21]:
L4 = []
%time tri_symmetric_search_mat4(T, k, d, L4, counts=[1, 1, 0, 0, 0], bound = 11)

CPU times: user 2min 31s, sys: 1.08 s, total: 2min 32s
Wall time: 2min 30s


### Note
The frobenius-fixed search finds a decomposition of length 11.

### Matrices with $\mathbb F_{3^6}$

In [89]:
p, n = 3, 6
k = GF(p^n)
S = MatrixSpace(GF(p), n, n)
d = make_dict_global_mat(k, S)
T = multiplication_formula_mat(k, S)

# Sandbox

*Delete after use!*

In [47]:
def extend_solutions(sols):
    extended_sols=copy(sols)
    for sol in sols:
        orbit = list_frobenius_orbit(sol)
        for elem in orbit:
            if not elem in extended_sols:
                extended_sols.append(elem)
    return extended_sols

def spectrum(sol):
    field = parent(sol[0][0])
    deg = field.degree()
    spectrum = deg*[0]
    for a, b in sol:
        spectrum[first_nonzero_coord(a)] += 1
    return spectrum

In [48]:
p, n = 3, 5
k = GF(p^n)
V = VectorSpace(GF(p), n^2)
d = make_dict_global(k, V)
T = multiplication_formula(k, V)

In [49]:
def describe_ranks(T, sol, d):
    S = copy(T)
    for a, l in sol:
        P = a.polynomial()
        for j in range(P.degree()+1):
            S[j] -= P[j]*l*d[a]
        print [s.rank() for s in S]

In [50]:
def make_small_trisym_all(field, rank):
    base_field = field.base_ring()
    elems = []
    for j in range(field.degree()):
        elems += fields_elements(field, j)
    dictionnary = dict()
    S = MatrixSpace(base_field, field.degree(), field.degree())
    L = [S() for j in range(field.degree())]
    other_dic = make_dict_global_mat(field, S)
    _make_small_trisym_all(elems, rank, dictionnary, L, [], other_dic)
    return dictionnary
    
    
def _make_small_trisym_all(field_elements, count, dictionnary, current_sum, decomposition, other_dic):
    nonzero_elems = list(current_sum[0].base_ring())[1:]
    if count == 1:
        for x in field_elements:
            for l in nonzero_elems:
                new_decomposition = decomposition+[(x, l)]
                new_sum = copy(current_sum)
                P = x.polynomial()
                for i in range(0, P.degree()+1):
                    new_sum[i] += P[i]*l*other_dic[x]
                for i in range(0, len(new_sum)):
                    new_sum[i].set_immutable()
                    
                #if dictionnary.has_key(tuple(new_sum)):
                #    print "*"
                dictionnary[tuple(new_sum)] = copy(new_decomposition)
    elif count > 1:
        for j in range(len(field_elements)):
            x = field_elements[j]
            for l in nonzero_elems:
                new_decomposition = decomposition+[(x, l)]
                new_sum = copy(current_sum)
                P = x.polynomial()
                for i in range(0, P.degree()+1):
                    new_sum[i] += P[i]*l*other_dic[x]
                new_sum2 = copy(new_sum)
                for i in range(0, len(new_sum)):
                    new_sum[i].set_immutable()
                dictionnary[tuple(new_sum)] = copy(new_decomposition)
                _make_small_trisym_all(field_elements[j+1:], count-1, dictionnary, new_sum2, new_decomposition, other_dic)              

In [51]:
def make_small_trisym_coord(field, rank, other_dic = None, j = 0):
    base_field = field.base_ring()
    elems = fields_elements(k, j)

    dictionnary = dict()
    S = MatrixSpace(base_field, field.degree(), field.degree())
    if other_dic == None:
        other_dic = make_dict_global_mat(field, S)
    current_sum = S()
    _make_small_trisym_first(elems, rank, dictionnary, current_sum, [], other_dic)
    zeromat = S()
    zeromat.set_immutable()
    dictionnary[zeromat] = [[]]
    return dictionnary
    
    
def _make_small_trisym_first(field_elements, count, dictionnary, current_sum, decomposition, other_dic):
    nonzero_elems = list(current_sum[0].base_ring())[1:]
    if count == 1:
        for x in field_elements:
            for l in nonzero_elems:
                new_decomposition = decomposition+[(x, l)]
                new_sum = current_sum+l*other_dic[x]
                new_sum.set_immutable()
                if dictionnary.has_key(new_sum):
                    dictionnary[new_sum] += [new_decomposition]
                else:
                    dictionnary[new_sum] = [new_decomposition]
                
    elif count > 1:
        for j in range(len(field_elements)):
            x = field_elements[j]
            for l in nonzero_elems:
                new_decomposition = decomposition+[(x, l)]
                new_sum = current_sum+l*other_dic[x]
                new_sum2 = copy(new_sum)
                new_sum.set_immutable()
                if dictionnary.has_key(new_sum):
                    dictionnary[new_sum] += [new_decomposition]
                else:
                    dictionnary[new_sum] = [new_decomposition]
                _make_small_trisym_first(field_elements[j+1:], count-1, dictionnary, new_sum2, new_decomposition, other_dic)              

In [52]:
def find_decomposition_mat_memo(t, B, d, L, count, bound, precomputed, G = [], debug = False):
    t.set_immutable()
    r = t.rank()
    #if debug:
    #    print count, bound, r
    precomputed_rank, precomputed_dict = precomputed
    Field = t.base_ring()
    nonZeroElems = list(Field)[1:]
    if r <= bound:
        if r <= precomputed_rank:
            if precomputed_dict.has_key(t):
                decomps = precomputed_dict[t]
                if debug:
                    print len(decomps)
                for dec in decomps:
                    if len(dec) <= bound:
                        G2 = G+dec #it should make a copy anyway
                        G2.sort()
                        if not G2 in L:
                            L.append(G2)
            if count > 0:
                for j in range(len(B)):
                    b = B[j]
                    B2 = B[j+1:]
                    for x in nonZeroElems:
                        t2 = t - x*d[b]
                        r2 = t2.rank()
                        if r2 == r: #else what happens if we allow the rank to rise? ie r2 > r?
                            find_decomposition_mat_memo(t2, B2, d, L, count-1, bound-1, precomputed, copy(G+[(b, x)]), debug)
        elif count == 0:
            elems = []
            for b in B:
                for x in nonZeroElems:
                    if (t - x*d[b]).rank() < r:
                        elems.append((b, x))
                        break
            for j in range(len(elems)):
                y = elems[j]
                B2 = [elems[i][0] for i in range(j+1, len(elems))]
                find_decomposition_mat_memo(t-y[1]*d[y[0]], B2, d, L, count, bound-1, precomputed, copy(G+[y]), debug)
        else:
            for j in range(len(B)):
                b = B[j]
                B2 = B[j+1:]
                for x in nonZeroElems:
                    t2 = t - x*d[b]
                    r2 = t2.rank()
                    if r2 < r:                    
                        find_decomposition_mat_memo(t2, B2, d, L, count, bound-1, precomputed, copy(G+[(b, x)]), debug)
                    elif r2 == r: #else what happens if we allow the rank to rise? ie r2 > r?
                        find_decomposition_mat_memo(t2, B2, d, L, count-1, bound-1, precomputed, copy(G+[(b, x)]), debug)

"""
Compute a list `Lglob` of tri-symmetric decompositions, where for each coordinate
the search is minimal.  
"""
def tri_symmetric_search_mat_memo(T, k, d, precomputed_all, Lglob, counts = None, bound = Infinity, j = 0, Lloc = [], debug = False):
    debug_before = debug
    z4 = gen(k)
    Ldebug = [(z4^2 + 2*z4 + 1, 1), (2*z4^2 + 1, 2), (z4^3 + 2*z4 + 1, 1), (2*z4^3 + 1, 2), (2*z4^3 + z4 + 1, 2), (2*z4^3 + 2*z4^2 + z4 + 1, 2)]

    if debug:
        print "tri-sym: begin"
        print T[j].rank(), bound
    if counts == None:
        counts = len(T)*[0]

    
        
    if finished(T):
        if debug:
            print "tri-sym: end"
        Lloc.sort()
        Lglob.append(Lloc)
    elif T[j].rank() <= bound:
        M = []
        count = counts[j] # min(count[j], bound) smth like that?
        find_decomposition_mat_memo(T[j], fields_elements(k, j), d, M, count, bound, precomputed_all[j], debug = debug)
        for m in M:
            if is_in(m, Ldebug) and m != []:
                debug = True
            else:
                debug = debug_before
                
            T2 = copy(T)
            T2[j] = 0
            Lloc2 = Lloc+m
            if debug:
                print Lloc2
            for x in m:
                P = x[0].polynomial()
                for l in range(j+1, P.degree()+1):
                    T2[l] = T2[l] - P[l]*x[1]*d[x[0]]
            
            tri_symmetric_search_mat_memo(T2, k, d, precomputed_all, Lglob, counts, bound-len(m), j+1, Lloc2, debug = debug)

In [53]:
%time dics = make_small_trisym_coord(k, 2, j=0)

CPU times: user 198 ms, sys: 9.21 ms, total: 207 ms
Wall time: 188 ms


In [24]:
len(dics)

13123

In [56]:
p, n, r = 3, 5, 2
k = GF(p^n)
S = MatrixSpace(GF(p), n, n)
d = make_dict_global_mat(k, S)
T = multiplication_formula_mat(k, S)
#dico = make_small_trisym(k, 3)
precomputed_all = [(r, make_small_trisym_coord(k, r, j=i)) for i in range(n)]

In [85]:
L = []
%time tri_symmetric_search_mat_memo(T, k, d, precomputed_all, L, counts=[2, 1, 0, 0], bound = 9)

[(z4^2 + 2*z4 + 1, 1), (2*z4^2 + 1, 2), (z4^3 + 2*z4 + 1, 1), (2*z4^3 + 1, 2), (2*z4^3 + z4 + 1, 2), (2*z4^3 + 2*z4^2 + z4 + 1, 2)]
tri-sym: begin
3 3
2
[(z4^2 + 2*z4 + 1, 1), (2*z4^2 + 1, 2), (z4^3 + 2*z4 + 1, 1), (2*z4^3 + 1, 2), (2*z4^3 + z4 + 1, 2), (2*z4^3 + 2*z4^2 + z4 + 1, 2), (z4^3 + z4^2 + z4, 1), (z4^3 + 2*z4^2 + z4, 2), (2*z4^3 + z4^2 + z4, 1)]
tri-sym: begin
0 0
tri-sym: end
[(z4^2 + 2*z4 + 1, 1), (2*z4^2 + 1, 2), (z4^3 + 2*z4 + 1, 1), (2*z4^3 + 1, 2), (2*z4^3 + z4 + 1, 2), (2*z4^3 + 2*z4^2 + z4 + 1, 2), (z4^2 + z4, 2), (2*z4^2 + z4, 1), (2*z4^3 + 2*z4^2 + z4, 1)]
tri-sym: begin
1 0
CPU times: user 2.6 s, sys: 22.3 ms, total: 2.63 s
Wall time: 2.62 s


In [59]:
%time mdc = make_small_trisym_coord(k, 3, j=0);

CPU times: user 12.2 s, sys: 131 ms, total: 12.3 s
Wall time: 12.2 s


In [86]:
len(L), len(L4)

(23, 15)

In [75]:
L4 = []
%time tri_symmetric_search_mat4(T, k, d, L4, counts=[2, 1, 0, 0], bound = 9)

CPU times: user 2.49 s, sys: 55.6 ms, total: 2.55 s
Wall time: 2.5 s


In [79]:
f

[(z4^2 + 2*z4 + 1, 1),
 (2*z4^2 + 1, 2),
 (z4^3 + 2*z4 + 1, 1),
 (2*z4^3 + 1, 2),
 (2*z4^3 + z4 + 1, 2),
 (2*z4^3 + 2*z4^2 + z4 + 1, 2)]

In [78]:
T[0] == sum_fn(f, d)

True

In [38]:
for l in L4:
    print l in L, spectrum(l)

True [4, 2, 2, 1]
True [6, 3, 0, 0]
True [4, 4, 1, 0]
True [4, 3, 1, 1]
True [4, 4, 1, 0]
True [6, 1, 2, 0]
True [6, 2, 1, 0]
True [6, 1, 1, 1]
True [5, 2, 2, 0]
True [6, 1, 2, 0]
True [5, 1, 2, 1]
True [4, 4, 0, 1]
True [5, 3, 0, 1]
True [5, 1, 2, 1]
True [5, 3, 1, 0]


In [31]:
for l in L:
    print l in L4, spectrum(l)

True [4, 2, 2, 1]
True [6, 3, 0, 0]
True [4, 4, 1, 0]
True [4, 3, 1, 1]
True [4, 4, 1, 0]
True [6, 1, 2, 0]
True [6, 2, 1, 0]
True [6, 1, 1, 1]
True [5, 2, 2, 0]
False [6, 3, 0, 0]
False [6, 1, 2, 0]
True [6, 1, 2, 0]
True [5, 1, 2, 1]
False [6, 1, 1, 1]
False [6, 3, 0, 0]
True [4, 4, 0, 1]
True [5, 3, 0, 1]
False [6, 1, 1, 1]
True [5, 1, 2, 1]
False [6, 3, 0, 0]
False [6, 3, 0, 0]
False [6, 3, 0, 0]
True [5, 3, 1, 0]


In [41]:
[L[-2][x] for x in [0, 1, 2, 5, 6, -1]]

[(z4^2 + 2*z4 + 1, 1),
 (2*z4^2 + 1, 2),
 (z4^3 + 2*z4 + 1, 1),
 (2*z4^3 + 1, 2),
 (2*z4^3 + z4 + 1, 2),
 (2*z4^3 + 2*z4^2 + z4 + 1, 2)]

In [170]:
LL4 = extend_solutions(L4)

In [171]:
len(LL), len(LL4)

(39, 39)

In [172]:
LL == LL4

False

In [69]:
dic[b[3]]

[[(z4^3 + 1, 1), (2*z4^3 + z4^2 + 2*z4 + 1, 2), (z4^3 + 2*z4 + 1, 2)],
 [(2*z4^2 + 2*z4 + 1, 1), (2*z4^2 + 1, 2), (2*z4^3 + z4^2 + 1, 2)]]

In [63]:
def sum_fn(L, d, j = None):
    s = L[0][0].parent()()
    if j == None:
        for l in L:
            s += l[1] * d[l[0]]
    else:
        for l in L:
            s += l[1] * l[0].polynomial()[j] * d[l[0]]

    return s

def check_solution(L, T, d):
    for j in range(len(T)):
        if T[j] != sum_fn(L, d, j):
            return False
    return True

def check_all_solutions(L, T, d):
    for l in L:
        if not check_solution(l, T, d):
            return False
    return True

In [50]:
z4 = gen(k)

In [51]:
f = [(z4^2 + 2*z4 + 1, 1), (2*z4^2 + 1, 2), (z4^3 + 2*z4 + 1, 1), (2*z4^3 + 1, 2), (2*z4^3 + z4 + 1, 2), (2*z4^3 + 2*z4^2 + z4 + 1, 2)]

In [52]:
f2 = copy(f)

In [53]:
f.sort()

In [54]:
f == f2

True

In [55]:
f

[(z4^2 + 2*z4 + 1, 1),
 (2*z4^2 + 1, 2),
 (z4^3 + 2*z4 + 1, 1),
 (2*z4^3 + 1, 2),
 (2*z4^3 + z4 + 1, 2),
 (2*z4^3 + 2*z4^2 + z4 + 1, 2)]

In [56]:
f2

[(z4^2 + 2*z4 + 1, 1),
 (2*z4^2 + 1, 2),
 (z4^3 + 2*z4 + 1, 1),
 (2*z4^3 + 1, 2),
 (2*z4^3 + z4 + 1, 2),
 (2*z4^3 + 2*z4^2 + z4 + 1, 2)]

In [57]:
for x in k:
    if x in f:
        print x

In [38]:
def is_in(a, b):
    for x in a:
        if x not in b:
            return False
    return True

In [62]:
is_in(L4, L)

True

In [90]:
describe_ranks(T, f, d)

[4, 4, 4, 4]
[4, 4, 4, 4]
[3, 4, 4, 4]
[2, 4, 4, 4]
[1, 4, 4, 4]
[0, 3, 3, 3]


In [91]:
describe_ranks(T, [(z4^2 + 2*z4 + 1, 1), (2*z4^2 + 1, 2), (z4^3 + 2*z4 + 1, 1), (2*z4^3 + 1, 2), (2*z4^3 + z4 + 1, 2), (2*z4^3 + 2*z4^2 + z4 + 1, 2)], d)

[4, 4, 4, 4]
[4, 4, 4, 4]
[3, 4, 4, 4]
[2, 4, 4, 4]
[1, 4, 4, 4]
[0, 3, 3, 3]


In [107]:
t = sum_fn([(z4^2 + 2*z4 + 1, 1), (2*z4^2 + 1, 2)], d)+d[z4^3 + 2*z4 + 1]+2*d[2*z4^3 + 1]

In [109]:
(T[0]-t-2*d[2*z4^3 + z4 + 1]).rank()

1

### Questions

- Pourquoi y a-t-il des éléments dans `L` qui ne sont pas trouvés par `L4` ?
  - Est-ce à cause de l'élimination au fure et à mesure de l'algo 4 ?
  - Est-ce que c'est pour ça qu'on ne trouve pas de solution dans GF(3^6) ?

In [110]:
p, n, r = 3, 5, 4
k = GF(p^n)
S = MatrixSpace(GF(p), n, n)
d = make_dict_global_mat(k, S)
T = multiplication_formula_mat(k, S)
#dico = make_small_trisym(k, 3)
#%time precomputed_all = [(r, make_small_trisym_coord(k, r, j=i)) for i in range(n)]

In [133]:
L = []
%time tri_symmetric_search_mat_memo(T, k, d, precomputed_all, L, counts=[2, 2, 2, 2, 2], bound = 11)

CPU times: user 1h 11min 30s, sys: 56.5 s, total: 1h 12min 27s
Wall time: 1h 12min 26s


In [116]:
L4 = []
%time tri_symmetric_search_mat4(T, k, d, L4, counts=[0, 0, 0, 0, 0], bound = 9)

CPU times: user 1.21 s, sys: 11.9 ms, total: 1.23 s
Wall time: 1.19 s


In [113]:
L4

[]

In [69]:
z3 = gen(k)
f = [(2*z3^2 + 1, 1),
  (z3 + 1, 1),
  (2*z3 + 1, 2)]

In [64]:
sum_fn(f, d)

[0 0 0]
[0 1 2]
[0 2 0]

In [81]:
j = 1
t = T[j] - sum_fn(f, d, j)
t

[0 1 0]
[1 1 1]
[0 1 0]

In [80]:
t - d[z3^2 + z3] - 2*d[2*z3^2+z3]

[0 2 0]
[2 1 2]
[0 2 0]

In [24]:
d[2*x^2+2*x+2]

[1 1 2]
[1 1 2]
[2 2 1]

In [20]:
T

[
[1 0 0]  [0 1 0]  [0 0 1]
[0 0 2]  [1 0 1]  [0 1 0]
[0 2 0], [0 1 2], [1 0 1]
]

In [23]:
fields_elements(k, 2)

[z3^2]

In [105]:
x = gen(k)

In [106]:
Lj1200 = [[(1, 1), (x+1, 2), (x^3+x^2+2*x+1, 1), (2*x^3+x+1, 2), (2*x^3+2*x^2+2*x+1, 1), (x^3+2*x^2+x, 1), (x^2, 2), (x^3+x^2, 2), (x^3, 2)],                  
 [(1, 2), (x^2+x+1, 1), (x^3+2*x^2+1, 2), (x^3+2*x^2+2*x+1, 2), (x, 2), (2*x^2+x, 2), (2*x^3+x, 2), (2*x^3+2*x^2+x, 1), (x^3, 2)],                    
 [(x+1, 2), (x^2+1, 1), (2*x^2+x+1, 1), (2*x^2+2*x+1, 2), (x^3+2*x^2+x+1, 1), (x^2+x, 1), (x^3+x, 2), (2*x^3+2*x^2+x, 1), (x^3, 1)],                  
 [(2*x+1, 2), (x^2+2*x+1, 1), (2*x^2+1, 1), (2*x^2+2*x+1, 2), (x^3+2*x^2+x+1, 1), (x^2+x, 1), (x^3+x^2+x, 2), (2*x^3+x^2+x, 1), (2*x^3+x^2, 2)],      
 [(x^2+x+1, 2), (2*x^2+1, 2), (2*x^2+x+1, 2), (x^3+2*x+1, 2), (x^3+x^2+1, 2), (2*x^2+x, 1), (x^2, 2), (2*x^3+x^2, 2), (x^3, 1)],                      
 [(x^2+2*x+1, 2), (2*x^2+1, 1), (x^3+x+1, 2), (x^3+x^2+x+1, 2), (x^2+x, 1), (x^3+x^2+x, 2), (2*x^3+x, 2), (2*x^3+2*x^2+x, 2), (2*x^3+x^2, 2)],        
 [(x^3+1, 2), (x^3+2*x^2+1, 2), (2*x^3+x^2+x+1, 1), (2*x^3+2*x^2+2*x+1, 2), (x, 2), (x^3+x, 2), (x^2, 2), (x^3+x^2, 1), (x^3, 2)],                    
 [(x^3+x^2+1, 1), (2*x^3+1, 2), (2*x^3+x+1, 1), (2*x^3+2*x+1, 2), (2*x^3+2*x^2+1, 1), (x^2+x, 2), (x^3+x, 2), (x^3+x^2, 2), (2*x^3+x^2, 2)],          
 [(x^3+x^2+1, 1), (2*x^3+x+1, 2), (2*x^3+2*x^2+x+1, 2), (2*x^3+2*x^2+2*x+1, 2), (x, 1), (2*x^2+x, 1), (x^3+x^2+x, 2), (2*x^3+x, 1), (2*x^3+x^2+x, 2)],
 [(x^3+x^2+x+1, 2), (x^3+x^2+2*x+1, 2), (2*x^3+x+1, 2), (2*x^3+2*x^2+1, 1), (x, 2), (x^3+2*x^2+x, 1), (2*x^3+2*x^2+x, 1), (x^2, 2), (x^3, 2)],        
 [(x^3+x^2+x+1, 2), (x^3+x^2+2*x+1, 2), (2*x^3+x+1, 2), (2*x^3+2*x^2+1, 1), (x^2+x, 2), (x^3+x, 2), (x^3+2*x^2+x, 2), (2*x^3+x^2+x, 1), (x^3+x^2, 2)]]


In [107]:
for l in Lj:
    l.sort()

In [109]:
for l in L4:
    print l in Lj

True
True
True
True
True
True
True
True
True
True
True


## More algebras: truncated polynomials

In [19]:
def pol_to_bil(P):
    n = P.parent().modulus().degree()
    S = MatrixSpace(base_ring(P), n, 1)
    s = S([P[j] for j in range(n-1, -1, -1)])
    return s*s.transpose()

def make_dict_global_pol(S):
    d = dict()
    for s in S:
        d[s] = pol_to_bil(s)
    return d

def multiplication_formula_pol(S):
    L = []
    n = S.modulus().degree()
    M = MatrixSpace(base_ring(S), n, n)
    for j in range(n):
        m = M()
        for k in range(j+1):
            m[j-k, k] = 1
        L.append(m)
    return L

In [20]:
def pol_elements(S, j):
    L = []
    for y in S:
        if y[j] == 1:
            boo = True
            for l in range(j):
                if y[l] != 0:
                    boo = False
                    break
            if boo:
                L.append(y)
    return L

In [21]:
def find_decomposition_pol(t, B, d, L, count, bound, G = []):
    r = t.rank()
    if r <= bound:
        if r == 0:
            L.append(G)
        elif count == 0:
            Field = t.base_ring()
            #nonZeroElems = [x for x in Field if x != 0]
            nonZeroElems = list(Field)[1:]
            elems = []
            for b in B:
                for x in nonZeroElems:
                    if (t - x*d[b]).rank() < r:
                        elems.append((b, x))
                        break
            for j in range(len(elems)):
                y = elems[j]
                B2 = [elems[i][0] for i in range(j+1, len(elems))]
                find_decomposition_pol(t-y[1]*d[y[0]], B2, d, L, count, bound-1, copy(G+[y]))
        else:
            Field = t.base_ring()
            #nonZeroElems = [x for x in Field if x != 0]
            nonZeroElems = list(Field)[1:]
            for j in range(len(B)):
                b = B[j]
                B2 = B[j+1:]
                for x in nonZeroElems:
                    t2 = t - x*d[b]
                    r2 = t2.rank()
                    if r2 < r:                    
                        find_decomposition_pol(t2, B2, d, L, count, bound-1, copy(G+[(b, x)]))
                    else:#if r2 == r: #else what happens if we allow the rank to rise? ie r2 > r?
                        find_decomposition_pol(t2, B2, d, L, count-1, bound-1, copy(G+[(b, x)]))

"""
Compute a list `Lglob` of tri-symmetric decompositions, where for each coordinate
the search is minimal.  
"""
def tri_symmetric_search_pol(T, S, d, Lglob, counts = None, bound = Infinity, j = 0, Lloc = []):

    if counts == None:
        counts = len(T)*[0]
        
    if finished(T):
        #Lloc.sort()
        Lglob.append(Lloc)
    elif T[j].rank() <= bound:
        M = []
        count = counts[j] # min(count[j], bound) smth like that?
        find_decomposition_pol(T[j], pol_elements(S, j), d, M, count, bound)

        for m in M:
            T2 = copy(T)
            T2[j] = 0
            Lloc2 = Lloc+m
            for x in m:
                P = x[0]
                for l in range(j+1, P.lift().degree()+1):
                    T2[l] = T2[l] - P[l]*x[1]*d[x[0]]
            
            tri_symmetric_search_pol(T2, S, d, Lglob, counts, bound-len(m), j+1, Lloc2)

In [22]:
R.<x> = GF(3)['x']

In [23]:
S.<y> = PolynomialQuotientRing(R, x^2)

In [11]:
S.

Univariate Quotient Polynomial Ring in y over Finite Field of size 3 with modulus x^2

In [19]:
base_ring(y)

Finite Field of size 3

In [22]:
S.modulus().degree()

2

In [23]:
y.parent()

Univariate Quotient Polynomial Ring in y over Finite Field of size 3 with modulus x^2

In [46]:
M = pol_to_bil(S(1)).parent()

In [61]:
M([0, 1, 1, 0])

[0 1]
[1 0]

In [68]:
multiplication_formula_pol(S)

[
[1 0]  [0 1]
[0 0], [1 0]
]

In [37]:
range(4-1, -1, -1)

[3, 2, 1, 0]

In [75]:
pol_elements(S, 1)

[y]

In [45]:
p, n = 5, 3
R.<x> = GF(p)['x']
S.<y> = PolynomialQuotientRing(R, x^n)
d = make_dict_global_pol(S)
T = multiplication_formula_pol(S)

In [46]:
L = []
%time tri_symmetric_search_pol(T, S, d, L, counts=[2, 2, 2], bound = 5)

CPU times: user 15.4 s, sys: 107 ms, total: 15.5 s
Wall time: 15.2 s


In [49]:
len(L)

21

In [50]:
L

[[(1, 2), (y^2 + 1, 2), (3*y^2 + 1, 1), (y^2 + y, 3), (4*y^2 + y, 2)],
 [(1, 2), (2*y^2 + 1, 1), (4*y^2 + 1, 2), (2*y^2 + y, 4), (3*y^2 + y, 1)],
 [(1, 4),
  (y^2 + y + 1, 4),
  (y^2 + 4*y + 1, 4),
  (4*y^2 + 2*y + 1, 4),
  (4*y^2 + 3*y + 1, 4)],
 [(y + 1, 2), (y^2 + y + 1, 2), (3*y^2 + y + 1, 1), (y, 1), (4*y^2 + y, 4)],
 [(y + 1, 3),
  (3*y^2 + y + 1, 3),
  (4*y^2 + y + 1, 4),
  (y^2 + y, 2),
  (3*y^2 + y, 3)],
 [(2*y + 1, 2),
  (2*y^2 + 2*y + 1, 1),
  (4*y^2 + 2*y + 1, 2),
  (y, 3),
  (3*y^2 + y, 2)],
 [(2*y + 1, 3),
  (y^2 + 2*y + 1, 4),
  (2*y^2 + 2*y + 1, 3),
  (y^2 + y, 4),
  (2*y^2 + y, 1)],
 [(3*y + 1, 2),
  (2*y^2 + 3*y + 1, 1),
  (4*y^2 + 3*y + 1, 2),
  (y, 2),
  (2*y^2 + y, 3)],
 [(3*y + 1, 3),
  (y^2 + 3*y + 1, 4),
  (2*y^2 + 3*y + 1, 3),
  (3*y^2 + y, 4),
  (4*y^2 + y, 1)],
 [(4*y + 1, 2),
  (y^2 + 4*y + 1, 2),
  (3*y^2 + 4*y + 1, 1),
  (y, 4),
  (y^2 + y, 1)],
 [(4*y + 1, 3),
  (3*y^2 + 4*y + 1, 3),
  (4*y^2 + 4*y + 1, 4),
  (2*y^2 + y, 2),
  (4*y^2 + y, 3)],
 [(y^2 + 1,

## Other algorithm

Closer to BDEZ algo

In [27]:
def kron(e, x):
    n = e.parent().dimension()
    V = VectorSpace(e.parent().base_ring(), n^3)
    v = V()
    for j in range(n):
        for i in range(n^2):
            v[j*n^2 + i] = e[j]*x[i]
            
    return v

def make_trisym_dict(k, S):
    elems = []
    for j in range(k.degree()):
        elems += fields_elements(k, j)
        
    d = dict()
    for x in elems:
        d[x] = kron(vector(x), vector(trace_to_bil_mat(x, S)))
    
    return d

def tensor_multiplication_formula(k, S):
    T = multiplication_formula_mat(k, S)
    E = VectorSpace(k.base_ring(), k.degree()).basis()
    T2 = kron(E[0], vector(T[0]))

    for j in range(1, k.degree()):
        T2 += kron(E[j], vector(T[j]))
    return T2

In [20]:
def rank_one_elements_tens(k):
    elems = []
    G = []
    for j in range(k.degree()):
        elems += fields_elements(k, j)
    for x in elems:
        G += [(x, kron(vector(x), vector(trace_to_bil_mat(x))))]
    
    return G

In [21]:
def expand_subspace_tensor(d, W, cpt = None, upto = Infinity, G = None, L=[]):
    if cpt <= upto:
        
        if G == None:
            G = d.values()

        if cpt == None:
            cpt = W.dimension()

        if W.dimension() == cpt and W.span([x for x in d.values() if x in W]).dimension() == cpt:
            L.append(W)
        else:

            for i in range(len(G)):
                g = G[i]
                if g in W:
                    continue
                G2 = G[i+1:]
                expand_subspace_tensor(d, W+W.span([g]), cpt+1, upto, G2, L)

In [22]:
def rank_one_basis_tensor(W, d):
    values = d.values()
    keys = d.keys()
    Bval = [w for w in W.basis() if w in values]
    Bkey = []
    for b in Bval:
        Bkey += [keys[values.index(b)]]
    if len(Bkey) == W.dimension():
        return Bkey
    else:
        for g in [x for x in values if x in W]:
            if g not in W.span(Bval):
                Bval += [g]
                Bkey += [keys[values.index(g)]]
        return Bkey, Bval
    
def formulas_tensor(t, W, d):
    Bkey, Bval = rank_one_basis_tensor(W, d)
    WW = t.parent().span_of_basis(Bval)
    coord = WW.coordinates(t)
    return [(Bkey[j], coord[j]) for j in range(len(coord))]

In [36]:
p, n = 3, 3
k = GF(p^n)
S = MatrixSpace(GF(p), n, n)
d = make_trisym_dict(k, S)
T = tensor_multiplication_formula(k, S)
W = T.parent().span([T])

In [103]:
L = []
%time expand_subspace_tensor(d, W, upto = 3, L = L)

CPU times: user 156 ms, sys: 542 µs, total: 157 ms
Wall time: 141 ms


In [104]:
formulas_tensor(T, L[0], d)

[(z2, 2), (1, 2), (2*z2 + 1, 2)]

In [76]:
%time 1-1

CPU times: user 29 µs, sys: 1e+03 ns, total: 30 µs
Wall time: 44.8 µs


0

In [48]:
VV.span_of_basis?

In [81]:
W.ambient_vector_space() == parent(G[3])

True

In [43]:
vector(s)

(2, 1, 1, 0)

In [48]:
S(list(vector(s)))

[2 1]
[1 0]

In [47]:
list(vector(s))

[2, 1, 1, 0]

In [194]:
kron(vector(e1), vector(s1))

(1, 1, 2, 0, 1, 1, 2, 0)

In [195]:
s1

[1 1]
[2 0]

In [62]:
d.values()

[(1, 2, 2, 1, 0, 0, 0, 0),
 (0, 0, 0, 0, 1, 0, 0, 0),
 (0, 0, 0, 1, 0, 0, 0, 1),
 (1, 1, 1, 1, 2, 2, 2, 2)]

In [181]:
v.parent().dimension()

4

In [164]:
vector(e1)

(1, 0)

In [35]:
l = [2, 6, 5]

In [37]:
l.index(6)

1

In [84]:
d

{1: (1, 2, 2, 1, 0, 0, 0, 0),
 z2: (0, 0, 0, 0, 1, 0, 0, 0),
 z2 + 1: (0, 0, 0, 1, 0, 0, 0, 1),
 2*z2 + 1: (1, 1, 1, 1, 2, 2, 2, 2)}

In [58]:
SX = MatrixSpace(GF(3), 27, 13)

In [59]:
A = SX()

In [60]:
for j in range(A.ncols()):
    A[:,j] = d.values()[j]

In [64]:
A.

In [45]:
t = MatrixSpace(GF(3), n^3, 1)()

In [46]:
t[:, 0] = T

In [49]:
A.solve_right??

In [105]:
A.solve_right?

In [101]:
d.keys()

[1, z2, z2 + 1, 2*z2 + 1]

In [110]:
(9-1)/(3-1)

4

In [23]:
def solve_tensor(k):
    p = k.characteristic()
    n = k.degree()
    
    S = MatrixSpace(GF(p), n, n)
    d = make_trisym_dict(k, S)
    values = d.values()
    T = tensor_multiplication_formula(k, S)
    
    SX = MatrixSpace(GF(p), n^3, (p^n - 1)/(p - 1))
    A = SX()
    for j in range(A.ncols()):
        A[:, j] = values[j]
    S1 = MatrixSpace(GF(p), n^3, 1)
    B = S1()
    B[:, 0] = T
    return A.solve_right(B)

def nonzero_entries(v):
    cpt = 0
    for j in range(v.nrows()):
        if v[j, 0] != 0:
            cpt += 1
    return cpt

In [56]:
%time rep = solve_tensor(GF(3^3))

CPU times: user 17.6 ms, sys: 0 ns, total: 17.6 ms
Wall time: 21.5 ms


In [57]:
nonzero_entries(rep)

8

2

In [126]:
len(d.values()[0])

27

27

In [134]:
show(rep)

40 x 1 dense matrix over Finite Field of size 3 (use the '.str()' method to see the entries)

In [26]:
make_trisym_dict(GF(3^5), MatrixSpace(GF(3), 5, 5))

TypeError: unsupported operand type(s) for ** or pow(): 'function' and 'int'

## Trying to prove trisymmetric complexity

We try to prove that $\mu_q(3) = 5$ when $q>3$ is not a power of $2$ and $q = 1 \mod 3$. Helps to use a computer.

In that case we know that $\mathbb F_{q^3} = \mathbb F[x]/(x^3-\zeta)$ where $\zeta$ is not a cube.


In [233]:
#p, n = 7, 3
n = 3
#k = GF(p^n)
Kx = FractionField(QQ['x'], 'x')
S = MatrixSpace(Kx, n, n)
S1 = MatrixSpace(Kx, n, 1)
#d = make_trisym_dict(Kx.base_ring(), S)
T = zeta_multiplication_formula(S)
#W = T.parent().span([T])

In [294]:
def antidiag(S, j):
    s = S()
    n = S.ncols()
    for i in range(j+1):
        if j-i < n and i < n:
            s[j-i, i] = 1
    return s

def zeta_multiplication_formula(S):
    n = S.ncols()
    z = S.base_ring().gen()
    E = VectorSpace(S.base_ring(), n).basis()
    D = antidiag(S, 0)+z*antidiag(S, n)
    T = kron(E[0], vector(D))
    for j in range(1, n):
        D = antidiag(S, j)+z*antidiag(S, n+j)
        T += kron(E[j], vector(D))
    return T

def f(S, v):
    n = S.ncols()
    z = S.base_ring().gen()
    F = n * (antidiag(S, 0) + z*antidiag(S, n))

    return kron(vector(v), vector((F*v)*(F*v).transpose()))

def check_subsets(val, n, l, cpt, B, L):
    if len(L) == l:
        Sn = MatrixSpace(S.base_ring(), n^3, l)
        A = Sn()
        rep = None
        for j in range(l):
            A[:, j] = L[j]
        try:
            rep = L, A.solve_right(B)
            sucess = True
        except:
            sucess = False
        return sucess, rep
    else:
        for j in range(len(val)):
            c = check_subsets(val[j+1:], n, l, cpt+1, B, copy(L+[val[j]]))
            if c != None and c[0] == True:
                return c

In [ ]:
%time c = check_subsets(val, 3, 5, 0, MatrixSpace(Kx, n^3, 1)(T), [])

In [293]:
c

In [265]:
B

27 x 1 dense matrix over Fraction Field of Univariate Polynomial Ring in x over Rational Field (use the '.str()' method to see the entries)

In [178]:
Kx

Fraction Field of Univariate Polynomial Ring in x over Rational Field

In [194]:
s = S1((1, 1, -x^-1))

In [199]:
9^-1 * f(S, s)

(1, -1, x, -1, 1, -x, x, -x, x^2, 1, -1, x, -1, 1, -x, x, -x, x^2, -1/x, 1/x, -1, 1/x, -1/x, 1, -1, 1, -x)

In [295]:
val = []
for a in [1, 0]:
    for b in [-1, 0, 1, -x^-1, x^-1]:
        for c in [1, 0, 1, -x^-1, x^-1]:
            val += [(n^2)^-1 * f(S, S1((a, b, c)))]

In [296]:
len(val)

50

In [247]:
A = MatrixSpace(Kx, n^3, len(val)-1)()
i = 0
for j in range(len(val)-1):
    if val[j] == 0:
        i = 1
    A[:, j] = val[i+j]    

In [248]:
A

27 x 17 dense matrix over Fraction Field of Univariate Polynomial Ring in x over Rational Field (use the '.str()' method to see the entries)

In [249]:
B = 

In [278]:
nonzero_entries(A.solve_right(B))

7

In [277]:
A.solve_right(B)

[    2/(-2*x)]
[   -2/(-2*x)]
[           0]
[       -1/-x]
[ (-x + 1)/-x]
[           0]
[           0]
[           0]
[           0]
[       -1/-x]
[        2/-x]
[           0]
[(-x + 1)/x^2]
[           0]
[           0]
[           0]
[           0]

In [218]:
binomial(17, 5)

6188

In [259]:
MatrixSpace(Kx, n^3, 1).random_element().solve_right(B)

ValueError: matrix equation has no solutions

In [261]:
l = [3, 5, 6]

In [262]:
l[1:]

[5, 6]